In [ ]:
#Libs
import folium
import geopandas as gpd
import json
import geobr
from folium.plugins import Geocoder


In [ ]:
#Geobr 
ita = geobr.read_municipality(code_muni = 3131901)
ita_json = ita.to_json()

In [ ]:
#Reading json files

h = open('Data/Hidrografia/hidrografia1.geojson', encoding="utf8")
hidro = json.load(h)
pe = open('Data/Parque Ecológico/PE.geojson', encoding="utf8")
PE = json.load(pe)
app = open('Data/APP/app.geojson', encoding="utf8")
APP = json.load(app)
nascentes = open('Data/Nascentes/nascentes.geojson', encoding="utf8")
na = json.load(nascentes)
areas_verdes = open('Data/areasverdes/av.geojson', encoding="utf8")
av = json.load(areas_verdes)
app_na = open('Data/app_nascentes/app_nascentes.geojson', encoding="utf8")
App_na = json.load(app_na)

In [ ]:
#Reading shapefiles
risco_en = gpd.read_file('Data/risco enchente/risco_enchente.shp')
vocorocas = gpd.read_file('Data/Voçorocas/vocorocas.shp')
risco_geo = gpd.read_file('Data/Risco Geológico/risco_geo.shp')
zoneamento = gpd.read_file('Data/Zoneamento/zoneamento3.shp')
bairros = gpd.read_file('Data/Bairros/bairros.shp')
uc = gpd.read_file('Data/UC/uc.shp')
barragens = gpd.read_file('Data/Barragens/barragem.shp')

In [ ]:
#Data Processing
risco_en['AREA_M2'] = risco_en['AREA_M2'].astype(str)

vocorocas['AREA_M2'] = vocorocas['AREA_M2'].astype(str)

risco_geo['AREA_M2'] = risco_geo['AREA_M2'].astype(str)

zoneamento = zoneamento.to_crs(epsg=4326)

bairros = bairros.to_crs(epsg=4326)

uc = uc.to_crs(epsg=4326)

barragens = barragens.to_crs(epsg=4326)


In [ ]:
# Create a map using Folium
m = folium.Map(location=[-20.260629, -43.804297], zoom_start=10.5,show=False, control_scale = True )



In [ ]:
#Creating FeatureGroups

fg_voc = folium.FeatureGroup(name='Mapeamento Voçorocas (SEMAM)', show=False)
for _, r in vocorocas.iterrows():
  
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'brown', 'color':'black'})
    folium.Popup('Área (m²): ' + r['AREA_M2']).add_to(geo_j)
    fg_voc.add_child(geo_j)

fg_riscogeo = folium.FeatureGroup(name='Risco Geológico (Defesa Civil)', show=False)
for _, r in risco_geo.iterrows():
  
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'brown', 'color':'black'})
    folium.Popup('Área (m²): ' + r['AREA_M2']).add_to(geo_j)
    fg_riscogeo.add_child(geo_j)

fg_zon = folium.FeatureGroup(name='Zoneamento Urbano (SEMURB)', show=False)
for _, r in zoneamento.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange', 'color':'black'})
    folium.Popup('Setor: ' + r['SETOR']).add_to(geo_j)
    fg_zon.add_child(geo_j)

fg_bairros = folium.FeatureGroup(name='Bairros Municipais (IBGE)', show=False)
for _, r in bairros.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'yellow', 'color':'black'})
    folium.Popup('Bairro: ' + r['BAIRRO']).add_to(geo_j)
    fg_bairros.add_child(geo_j)

fg_uc = folium.FeatureGroup(name='Unidades de Conservação (Inst. Pristino)', show=False)
for _, r in uc.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green', 'color':'black'})
    folium.Popup( r['nome_uc']).add_to(geo_j)
    fg_uc.add_child(geo_j)

fg_barragem = folium.FeatureGroup(name='Barragens (FEAM)', show=False)
for _, r in barragens.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green', 'color':'black'})
    folium.Popup( r['nome_bar']).add_to(geo_j)
    fg_barragem.add_child(geo_j)

fg_av = folium.FeatureGroup(name='Áreas Verdes (SEMAM)', show=False,)
for feature in av['features']:
  geometry = feature["geometry"]["coordinates"]
  a = tuple(geometry)
  c = (a[1], a[0])
  fg_av.add_child(folium.Circle(location=(c),radius=10, weight=5, color='green' ))

fg = folium.FeatureGroup(name='Nascentes (SEMAM)', show=False,)
for feature in na['features']:
  geometry = feature["geometry"]["coordinates"]
  a =  tuple(geometry)
  b = tuple(a[0])
  c = (b[1], b[0])
  fg.add_child(folium.Circle(location=(c),radius=5, weight=3 ))

fg_riscoen = folium.FeatureGroup(name='Risco de Inundação (Defesa Civil)', show=False)

for _, r in risco_en.iterrows():
  
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'brown', 'color':'black'})
    folium.Popup('Área (m²): ' + r['AREA_M2']).add_to(geo_j)
    fg_riscoen.add_child(geo_j)


In [ ]:
# Add Layers to map
folium.GeoJson(data= ita_json, name='Limite Municipal(IBGE)',  style_function=lambda x: {'fillColor': '#000000', 'color': '#000000', 'weight': 5}).add_to(m)
folium.GeoJson(data= hidro, name='Hidrografia (IGAM adapt. SEMAM)', show=False, style_function=lambda x: {'fillColor': '#0000CD', 'color': '#0000CD', 'weight': 1} ).add_to(m)
fg.add_to(m)
folium.GeoJson(data= APP, name='Áreas de Preservação Permanentes - Hidrografia (SEMAM)', show=False,  style_function=lambda x: {'fillColor': '#000000', 'color': '	#76EEC6', 'weight':2} ).add_to(m)
folium.GeoJson(data= App_na, name='Áreas de Preservação Permanentes - Nascentes (SEMAM)', show=False,  style_function=lambda x: {'fillColor': '#000000', 'color': '	#76EEC6', 'weight':2} ).add_to(m)
fg_uc.add_to(m)  
folium.GeoJson(data= PE, name='Perímetro Parque Ecológico Municipal (SEMAM)', show=False,  style_function=lambda x: {'fillColor': '#000000', 'color': '	#3D9140', 'weight': 3} ).add_to(m)
fg_zon.add_to(m)  
fg_bairros.add_to(m) 
fg_barragem.add_to(m)
fg_voc.add_to(m) 
fg_riscogeo.add_to(m)
fg_riscoen.add_to(m)
fg_av.add_to(m)

In [ ]:
# Add Extra 
#Basemap Google
tiles = folium.TileLayer(
        tiles ='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = False,
        control = True,
        control_scale = True).add_to(m)

# Controle de camadas
folium.LayerControl().add_to(m)

# Medidor distancia
folium.plugins.MeasureControl(position='topleft').add_to(m)


# Barra de Pesquisa

Geocoder(collapsed=True, position='topleft').add_to(m)


In [ ]:
#salvando .html

m.save("teste.html")